In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

pd.set_option("display.max.columns", None)

In [69]:
# GET DATA
df = pd.read_csv(r"C:\Users\X1 Carbon\Desktop\IntroAI_Assignment2\processed_data.csv")
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,PhysHlth_Bin,MentHlth_Bin,EatHealthy
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0,5.0,1.0,1.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0,2.0,0.0,1.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0,1.0,0.0,1.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0,0.0,0.0,1.0


In [70]:
# SEPERATE PREDICTORS AND LABELS
X = df[["HighBP", "HighChol", "BMI", "GenHlth", "Age", "Income", "HeartDiseaseorAttack", "DiffWalk"]]
y = df["Diabetes_binary"]

X.shape, y.shape

((70692, 8), (70692,))

In [71]:
scaler = RobustScaler()
scale_cols = ["BMI", "Age", "Income", "GenHlth"]
X[scale_cols] = scaler.fit_transform(X[scale_cols])

X.head()

C:\Users\X1 Carbon\AppData\Local\Temp\ipykernel_20824\2265813040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[scale_cols] = scaler.fit_transform(X[scale_cols])


,HighBP,HighChol,BMI,GenHlth,Age,Income,HeartDiseaseorAttack,DiffWalk
0,1.0,0.0,-0.375,0.0,-1.25,0.5,0.0,0.0
1,1.0,1.0,-0.375,0.0,0.75,0.5,0.0,0.0
2,0.0,0.0,-0.375,-1.0,1.00,0.5,0.0,0.0
3,1.0,1.0,-0.125,0.0,0.50,0.5,0.0,0.0
4,0.0,0.0,0.000,-0.5,-0.25,0.5,0.0,0.0


In [72]:
# TEST NEAT PERFORMANCE 
def eval_genomes(genomes, config): 
  for genome_id, genome in genomes: 
    genome.fitness = 0 
    net = neat.nn.FeedForwardNetwork.create(genome, config)  
    for values, label in zip(X.values, y.values): 
      output = net.activate(values)
      pred = round(output[0]) 
      genome.fitness += (pred == label) 
    genome.fitness /= len(X) 

config = neat.Config(neat.DefaultGenome, 
                     neat.DefaultReproduction, 
                     neat.DefaultSpeciesSet, 
                     neat.DefaultStagnation, r"C:\Users\X1 Carbon\Desktop\IntroAI_Assignment2\config.txt") 
p = neat.Population(config)
#p = neat.Checkpointer.restore_checkpoint("neat-checkpoint-23") 
p.add_reporter(neat.StdOutReporter(True)) 
stats = neat.StatisticsReporter() 
p.add_reporter(stats) 
p.add_reporter(neat.Checkpointer(generation_interval=20)) 
best_model = p.run(eval_genomes, 150) 

# Display the winning genome. 
print('\nBest genome:\n{!s}'.format(best_model))


 ****** Running generation 0 ****** 

Population's average fitness: 0.49064 stdev: 0.10291
Best fitness: 0.72776 - size: (1, 8) - species 1 - id 6
Average adjusted fitness: 0.161
Mean genetic distance 1.449, standard deviation 0.376
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50      0.7    0.161     0
Total extinctions: 0
Generation time: 16.425 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.62529 stdev: 0.07632
Best fitness: 0.73485 - size: (1, 8) - species 1 - id 97
Average adjusted fitness: 0.216
Mean genetic distance 1.775, standard deviation 0.472
Population of 50 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    44      0.7    0.216     0
     2    0     6       --       --     0
Total extinctions: 0
Generation time: 15.517 sec (15.971 average)

 ****** Running generation 2 ****** 

Popul